# Homography estimation for OpenCV DoG implementation

### Performed on 4 datasets

* boat and bark datasets from https://www.robots.ox.ac.uk/~vgg/data/affine/
* pure rotations of the first image from "bark" by multiples of 90 degrees
* pure scaling of the first image from "bark" by multiples of 0.1 adjusted so that the aspect ratio is preserved

# Comparison of OpenCV DoG baseline and fixed version

#### using SIFT descriptor for matching

#### see https://github.com/opencv/opencv/pull/23124

```

Comparison of OpenCV baseline and fixed version:

Without fix:                                              With fix:


running experiment: synthetic rotation by multiples of pi/2

rotation        error     tentatives       inliers        rotation        error      tentatives        inliers
   90deg     0.499982           3464          3452           90deg     0.000383            3750           3742
  180deg     0.704858           3369          3349          180deg     0.000668            3659           3643
  270deg     0.499913           3438          3430          270deg     0.000971            3660           3649
     Sum     1.704754          10271         10231             Sum     0.002022           11069          11034

running experiment: bark

                error     tentatives       inliers                        error      tentatives        inliers
img2         1.332286            654           241            img2     1.128165             639            237
img3         2.178040            565           298            img3     1.688878             579            276
img4         1.339833            699           620            img4     0.960998             652            603
img5         0.711655            490           433            img5     0.711485             461            400
img6         1.256951            305           261            img6     1.004243             270            234
 Sum         6.818765           2713          1853             Sum     5.493769            2601           1750

running experiment: boat

                error     tentatives       inliers                        error      tentatives        inliers
img2         0.313058           2468           990            img2     0.353406            2242            892
img3         0.285631           1838           926            img3     0.160561            1729            903
img4         1.327328            757           241            img4     0.369467             721            220
img5         0.459227            550           140            img5     0.576069             547            144
img6         5.696269            267            63            img6     5.957721             270             52
 Sum         8.081513           5880          2360             Sum     7.417225            5509           2211

running experiment: synthetic rescaling by lanczos

scale           error     tentatives       inliers           scale        error      tentatives        inliers
0.2          0.283495            181           176             0.2     0.017970             158            154
0.3          0.261653            391           374             0.3     0.024879             334            290
0.4          0.212034            675           658             0.4     0.016976             632            590
0.5          0.173158           1059          1042             0.5     0.006069             968            951
0.6          0.142213           1402          1383             0.6     0.008504            1309           1281
0.7          0.116124           1784          1621             0.7     0.007711            1682           1651
0.8          0.069288           2315          2281             0.8     0.007902            2235           2087
0.9          0.037124           2663          2625             0.9     0.003026            2618           2579
Sum          1.295089          10470         10160             Sum     0.093035            9936           9583
```

In [1]:
from tqdm.notebook import tqdm

from dataset_utils import Hs_imgs_for_boat, Hs_imgs_for_bark, Hs_imgs_for_rotation, Hs_imgs_for_scaling
from opencv_utils import get_tentatives, get_visible_part_mean_absolute_reprojection_error


def compare_output(o1: str, o2: str):
    for line in zip(o1.split("\n"), o2.split("\n")):
        print(f"{line[0]}{line[1]:>60}")


class Printer:
    def __init__(self, print_output):
        self.print_output = print_output
        self.output = ""

    def print(self, s=""):
        if self.print_output:
            print(s)
        self.output = self.output + s + "\n"


def homography_estimation_experiment(detector, Hs_gt, imgs, e_name, instance_names, p=Printer(print_output=True),
                                     mean=True, sum=False):

    p.print()
    p.print(f"running experiment: {e_name}")
    p.print()

    kpts_0, desc_0 = detector.detectAndCompute(imgs[0], mask=None)
    p.print(f"reference img keypoints: {len(kpts_0)}")
    p.print()
    p.print(f"{'':>8}{'error':>12}{'keypoints':>12}{'tentatives':>12}{'inliers':>10}")

    sum_reproj_err = 0.0
    sum_keypoints = 0
    sum_tent_count = 0
    sum_in_count = 0

    for other_i in tqdm(range(1, len(imgs)), leave=False):
        kpts_other, desc_other = detector.detectAndCompute(imgs[other_i], mask=None)
        kpts_n = len(kpts_other)

        src_pts, dst_pts = get_tentatives(kpts_0, desc_0, kpts_other, desc_other, ratio_threshold=0.8)
        if len(src_pts) < 4:
            print(f"WARNING: less than 4 tentatives: {len(src_pts)}")
            continue

        H_est, inlier_mask = cv.findHomography(src_pts, dst_pts,
                                               cv.RANSAC,
                                               maxIters=100000,
                                               ransacReprojThreshold=0.5,
                                               confidence=0.9999)
        H_gt = Hs_gt[other_i - 1]


        reproj_err = get_visible_part_mean_absolute_reprojection_error(imgs[0], imgs[other_i], H_gt, H_est)
        tent_count = len(src_pts)
        in_count = int(inlier_mask.sum())

        p.print(f"{instance_names[other_i - 1]:>8}{reproj_err:>12.6f}{kpts_n:>12}{tent_count:>12}{in_count:>10}")
        sum_reproj_err += reproj_err
        sum_keypoints += kpts_n
        sum_tent_count += tent_count
        sum_in_count += in_count

    l = len(imgs) - 1
    if mean:
        p.print(f"{'Mean':>8}{sum_reproj_err/l:>12.6f}{sum_keypoints/l:>14.1f}{sum_tent_count/l:>12.1f}{sum_in_count/l:>10.1f}")
    if sum:
        p.print(f"{'Sum':>8}{sum_reproj_err:>12.6f}{sum_keypoints:>12}{sum_tent_count:>12}{sum_in_count:>10}")


Hs_boat, imgs_boat = Hs_imgs_for_boat()
Hs_boat_rot, imgs_boat_rot = Hs_imgs_for_boat(rotate_query_imgs=True)
Hs_bark, imgs_bark = Hs_imgs_for_bark()
Hs_bark_rot, imgs_bark_rot = Hs_imgs_for_bark(rotate_query_imgs=True)
Hs_rot, imgs_rot = Hs_imgs_for_rotation()
print("Exact scale adjustments:")
Hs_scaling, imgs_scaling, scales = Hs_imgs_for_scaling()


def run_on_descriptor(descriptor, name, print_output=True, mean=True, sum=True):
    p = Printer(print_output)
    p.print()
    p.print(f"descriptor: {name}:")
    p.print()
    homography_estimation_experiment(descriptor, Hs_rot, imgs_rot, "synthetic pi rotation", ["90deg", "180deg", "270deg"], p, mean, sum)
    homography_estimation_experiment(descriptor, Hs_bark, imgs_bark, "bark", [f"img{i}" for i in range(2, 7)], p, mean, sum)
    homography_estimation_experiment(descriptor, Hs_bark_rot, imgs_bark_rot, "bark rotated", [f"img{i}" for i in range(2, 7)], p, mean, sum)
    homography_estimation_experiment(descriptor, Hs_boat, imgs_boat, "boat", [f"img{i}" for i in range(2, 7)], p, mean, sum)
    homography_estimation_experiment(descriptor, Hs_boat_rot, imgs_boat_rot, "boat rotated", [f"img{i}" for i in range(2, 7)], p, mean, sum)
    homography_estimation_experiment(descriptor, Hs_scaling, imgs_scaling, "synthetic rescaling lanczos", [f"{s}" for s in scales], p, mean, sum)
    return None if print_output else p.output


Exact scale adjustments:
scale: 0.2 => 0.2, dimension: 765x510 => (153, 102)
scale: 0.3 => 0.2980392156862745, dimension: 765x510 => (228, 152)
scale: 0.4 => 0.4, dimension: 765x510 => (306, 204)
scale: 0.5 => 0.5019607843137255, dimension: 765x510 => (384, 256)
scale: 0.6 => 0.6, dimension: 765x510 => (459, 306)
scale: 0.7 => 0.6980392156862745, dimension: 765x510 => (534, 356)
scale: 0.8 => 0.8, dimension: 765x510 => (612, 408)
scale: 0.9 => 0.9019607843137255, dimension: 765x510 => (690, 460)


In [2]:
import cv2 as cv

descriptor = cv.SIFT_create()
run_on_descriptor(descriptor, "OpenCV SIFT")


descriptor: OpenCV SIFT:


running experiment: synthetic pi rotation

reference img keypoints: 3731

               error   keypoints  tentatives   inliers


  0%|          | 0/3 [00:00<?, ?it/s]

   90deg    0.499982        3715        3464      3452
  180deg    0.704858        3694        3369      3349
  270deg    0.499913        3721        3438      3430
    Mean    0.568251        3710.0      3423.7    3410.3
     Sum    1.704754       11130       10271     10231

running experiment: bark

reference img keypoints: 3731

               error   keypoints  tentatives   inliers


  0%|          | 0/5 [00:00<?, ?it/s]

    img2    1.332286        2973         654       241
    img3    2.178040        3935         565       298
    img4    1.339833        4774         699       620
    img5    0.711655        4510         490       433
    img6    1.256951        4678         305       261
    Mean    1.363753        4174.0       542.6     370.6
     Sum    6.818765       20870        2713      1853

running experiment: bark rotated

reference img keypoints: 3731

               error   keypoints  tentatives   inliers


...

    img2    0.313058        8545        2468       990
    img3    0.285631        6558        1838       926
    img4    1.327328        5269         757       241
    img5    0.459227        4932         550       140
    img6    5.696269        4257         267        63
    Mean    1.616303        5912.2      1176.0     472.0
     Sum    8.081513       29561        5880      2360

running experiment: boat rotated

reference img keypoints: 8849

               error   keypoints  tentatives   inliers


  0%|          | 0/5 [00:00<?, ?it/s]

    img2    0.574185        8478        2465       883
    img3    0.722569        6646        1845       959
    img4    1.509766        5268         767       222
    img5    0.591476        4897         568       153
    img6    6.130859        4256         262        58
    Mean    1.905771        5909.0      1181.4     455.0
     Sum    9.528854       29545        5907      2275

running experiment: synthetic rescaling lanczos

reference img keypoints: 3706

               error   keypoints  tentatives   inliers


  0%|          | 0/8 [00:00<?, ?it/s]

     0.2    0.283495         239         181       176
     0.3    0.261653         522         391       374
     0.4    0.212034         896         675       658
     0.5    0.173158        1402        1059      1042
     0.6    0.142213        1875        1402      1383
     0.7    0.116124        2409        1784      1621
     0.8    0.069288        2930        2315      2281
     0.9    0.037124        3470        2663      2625
    Mean    0.161886        1717.9      1308.8    1270.0
     Sum    1.295089       13743       10470     10160
